In [2]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

In [3]:
llm = ChatOpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)
math_agent = initialize_agent(tools,
                              llm,
                              agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [4]:
# some sample maths questions
# questions = [
#   "Find the square root of 5.4.",
#   "What is 3 divided by 7.34 raised to the power of pi?",
#   "What is the sin of 0.47 radians, divided by the cube root of 27?"
# ]

answer = math_agent.run("Find the square root of 5.4.")
answer

'The square root of 5.4 is approximately 2.32379000772445.'

In [66]:
def run_to_span(run, trace_id):
    return {
        'trace_id': trace_id,
        'span_id': str(run.id),
        'parent_id': str(run.parent_run_id) if run.parent_run_id else '',
        'name': str(run.name),
        "start_time_ms": int(run.start_time.timestamp() * 1000),
        "end_time_ms": int(run.end_time.timestamp() * 1000) if run.end_time else None,
        'error': str(run.error) if run.error else None,
        'inputs': run.inputs,
        'outputs': run.outputs,
        "attributes": None
    }

def run_to_spans(run, trace_id=None):
    spans = []
    if trace_id is None:
        trace_id = str(run.id)
    if run.child_runs:
        for child in run.child_runs:
            child_spans = run_to_spans(child, trace_id)
            spans += child_spans
    
    return spans + [run_to_span(run, trace_id)]

def run_to_trace(run):
    spans = run_to_spans(run)
    return {
        **spans[-1],
        "trace_spans": spans
    }

class CustomTracer(langchain.callbacks.tracers.base.BaseTracer):
    def __init__(self, stream) -> None:
        super().__init__()
        self._stream = stream
        
    def _persist_run(self, run) -> None:
        res=  run_to_trace(run)
        self._stream.log(res)

In [69]:
from weave.monitoring import StreamTable
import datetime
stream = StreamTable("timssweeney/experiments/lc_test_" + str(int(datetime.datetime.now().timestamp())))
tracer = CustomTracer(stream)
for i in range(10):
    answer = math_agent.run("Find the square root of 5.4.", callbacks=[tracer])
stream

In [ ]:
panel0.active_data["trace_spans"]